## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тестировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('../data/news_train.txt', sep='\t',header=None )
df_test = pd.read_csv('../data/news_test.txt', sep='\t',header=None )
df_train.columns = ['Рубрика', 'Заголовок', 'Текст']
df_test.columns = ['Рубрика', 'Заголовок', 'Текст']
df_train.head()

,Рубрика,Заголовок,Текст
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...


# 1. Обработать данные, получив для каждого текста набор токенов

In [3]:
from nltk import word_tokenize

df_train['Токены'] = df_train['Текст'].apply(word_tokenize)
df_test['Токены'] = df_test['Текст'].apply(word_tokenize)
df_train.head()

,Рубрика,Заголовок,Текст,Токены
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...,"[Нападающий, «, Вашингтон, Кэпиталз, », Алекса..."
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,...","[Власти, Мексики, объявили, подделкой, статую,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...,"[Южнокорейская, Samsung, анонсировала, защищен..."
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...,"[Контрольно-дисциплинарный, комитет, (, КДК, )..."
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...,"[Интернет-издание, Hopes, &, Fears, объявило, ..."


In [4]:
from nltk.stem.snowball import RussianStemmer
from nltk.corpus import stopwords

rs = RussianStemmer()
sw_r = set(stopwords.words('russian'))
sw_e = set(stopwords.words('english'))

def preprocessing(words):
    new_word_list = []
    for word in words:
        word = word.lower()
        if word.isalpha() and not (word in sw_r) and not (word in sw_e):
                new_word_list.append(rs.stem(word))
    return new_word_list


df_train['Токены'] = df_train['Токены'].apply(preprocessing)
df_test['Токены'] = df_test['Токены'].apply(preprocessing)

In [5]:
df_train.head()

,Рубрика,Заголовок,Текст,Токены
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...,"[напада, вашингтон, кэпиталз, александр, овечк..."
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,...","[власт, мексик, объяв, подделк, стат, май, про..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...,"[южнокорейск, samsung, анонсирова, защищен, ве..."
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...,"[комитет, кдк, рфс, снял, дисквалификац, полуз..."
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...,"[hopes, fears, объяв, сво, слиян, сайт, villag..."


# 2. Обучить word embeddings

In [18]:
from gensim.models import Word2Vec

tr = list(df_train['Токены'].values)
tst = list(df_test['Токены'].values)

w2v = Word2Vec(tr + tst, size=50 ,min_count=1, window=5, workers=1)

In [7]:
w2v.wv.most_similar(positive=['актер'])

[('актрис', 0.9336420297622681),
 ('режиссер', 0.9168128371238708),
 ('питт', 0.8717546463012695),
 ('макэв', 0.8695226311683655),
 ('крэйг', 0.8681154847145081),
 ('эджертон', 0.863692045211792),
 ('аффлек', 0.8572142720222473),
 ('джеймс', 0.8458917737007141),
 ('сценарист', 0.8441108465194702),
 ('карелл', 0.8436678051948547)]

In [23]:
w2v.wv.most_similar(positive=['законопроект'])

[('поправк', 0.9613997936248779),
 ('поправок', 0.9014572501182556),
 ('постановлен', 0.9008971452713013),
 ('закон', 0.8829703330993652),
 ('госдум', 0.8587325215339661),
 ('внесен', 0.8364836573600769),
 ('инициатив', 0.8321452140808105),
 ('законодательств', 0.8228006362915039),
 ('требован', 0.821120023727417),
 ('законодательн', 0.8087468147277832)]

In [9]:
w2v.wv.most_similar(positive=['генеральн'], negative=['директор'])

[('полномочн', 0.5840857028961182),
 ('альв', 0.5474891662597656),
 ('украинеофициальн', 0.5418031811714172),
 ('червиченк', 0.5274759531021118),
 ('хунт', 0.5258936882019043),
 ('штаатскапелл', 0.5244912505149841),
 ('ксир', 0.523795485496521),
 ('mlb', 0.5231428742408752),
 ('автопоезд', 0.5205785632133484),
 ('обветшан', 0.5083560943603516)]

In [10]:
w2v.wv.most_similar(positive=['журналист', 'учен'])

[('медик', 0.7839198112487793),
 ('исследовател', 0.7757724523544312),
 ('психолог', 0.7723990082740784),
 ('сноуд', 0.7616479992866516),
 ('хамз', 0.7600890398025513),
 ('долмат', 0.759471595287323),
 ('кузбасск', 0.7580829858779907),
 ('студент', 0.7569434642791748),
 ('признательн', 0.7506290078163147),
 ('коллег', 0.7470023036003113)]


# 3. Реализовать алгоритм классификации

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer=lambda x: x)
vectorizer.fit(list(df_train['Токены'].values) + list(df_test['Токены'].values))

idf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [27]:
import numpy as np

def to_vector(sentence):
    n = len(sentence)
    vector = np.zeros(50)
    for word in sentence:
        vector += idf[word] * w2v.wv[word]
    return vector / n

df_train['Вектор'] = df_train['Токены'].apply(to_vector)
df_test['Вектор'] = df_test['Токены'].apply(to_vector)

In [28]:
df_train.head()

,Рубрика,Заголовок,Текст,Токены,Вектор
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...,"[напада, вашингтон, кэпиталз, александр, овечк...","[0.9449942457694782, -2.8987251495099846, 2.59..."
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,...","[власт, мексик, объяв, подделк, стат, май, про...","[0.8885383913926918, -2.270422556930605, -0.08..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...,"[южнокорейск, samsung, анонсирова, защищен, ве...","[-0.5637279943209406, -7.860753782515256, -0.1..."
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...,"[комитет, кдк, рфс, снял, дисквалификац, полуз...","[0.33258345131283606, -3.3388640676564147, 0.9..."
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...,"[hopes, fears, объяв, сво, слиян, сайт, villag...","[-0.4890713593696506, -3.0280836876300934, -1...."


In [ ]:
X_train = [line for line in df_train['Вектор'].values]
y_train = df_train['Рубрика'].values

X_test = [line for line in df_test['Вектор'].values]
y_test = df_test['Рубрика'].values

## LogisticRegression

In [43]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(n_jobs=-1)
lg.fit(X_train, y_train)

(y_test == clf.predict(X_test)).mean()

0.829

## SVM

In [44]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)


(y_test == clf.predict(X_test)).mean()

0.829